In [1]:
%matplotlib inline
%store -r
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import mysql.connector
import scipy
import copy 
import json
from scipy import stats
from scipy.stats import skew
from math import sqrt
from numpy import mean, var
from sklearn import preprocessing
from datetime import datetime

In [2]:
df_kpi

,date,time,location,airline_operator,flight_no,route,aircraft_type,total_aboard,passengers_aboard,crew_aboard,...,category,brand,surviving_crew,survivors,year,month,day,decade,mortality_rate,survival_rate
0,1908-09-17,17:18:00,"Fort Myer, Virginia",Military - U.S. Army,,Demonstration,Wright Flyer III,2.0,1.0,1.0,...,military,Wright Brothers,1.0,1.0,1908-01-01,September,Thursday,1900,0.000000,1.000000
1,1909-09-07,17:18:00,"Juvisy-sur-Orge, France",,,Air show,Wright Byplane,1.0,0.0,1.0,...,non-military,Wright Brothers,1.0,0.0,1909-01-01,September,Tuesday,1900,0.000000,1.000000
2,1912-07-12,06:30:00,"Atlantic City, New Jersey",Military - U.S. Navy,,Test flight,Dirigible,5.0,0.0,5.0,...,military,other,0.0,0.0,1912-01-01,July,Friday,1910,1.000000,0.000000
3,1913-08-06,06:30:00,"Victoria, British Columbia, Canada",Private,,,Curtiss seaplane,1.0,0.0,1.0,...,non-military,other,0.0,0.0,1913-01-01,August,Wednesday,1910,1.000000,0.000000
6,1915-03-05,01:00:00,"Tienen, Belgium",Military - German Navy,,,Zeppelin L-8 (airship),41.0,0.0,41.0,...,military,other,24.0,24.0,1915-01-01,March,Friday,1910,0.414634,0.585366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5002,2021-03-02,17:05:00,"Pieri, Sudan",South Sudan Supreme Airlines,,Pieri - Yuai,Let L-410UVP-E,10.0,8.0,2.0,...,non-military,other,0.0,0.0,2021-01-01,March,Tuesday,2020,1.000000,0.000000
5003,2021-03-28,18:35:00,"Near Butte, Alaska",Soloy Helicopters,,Sightseeing Charter,Eurocopter AS350B3 Ecureuil,6.0,5.0,1.0,...,non-military,other,0.0,1.0,2021-01-01,March,Sunday,2020,1.000000,0.000000
5004,2021-05-21,18:00:00,"Near Kaduna, Nigeria",Military - Nigerian Air Force,,,Beechcraft B300 King Air 350i,11.0,7.0,4.0,...,military,Beechcraft,0.0,0.0,2021-01-01,May,Friday,2020,1.000000,0.000000
5005,2021-06-10,08:00:00,"Near Pyin Oo Lwin, Myanmar",Military - Myanmar Air Force,,Naypyidaw - Anisakan,Beechcraft 1900D,14.0,12.0,2.0,...,military,Beechcraft,1.0,2.0,2021-01-01,June,Thursday,2020,0.500000,0.500000


In [3]:
df_mysql = df_kpi

In [4]:
df_mysql.shape

(4770, 26)

In [5]:
df_mysql.head(1)

,date,time,location,airline_operator,flight_no,route,aircraft_type,total_aboard,passengers_aboard,crew_aboard,...,category,brand,surviving_crew,survivors,year,month,day,decade,mortality_rate,survival_rate
0,1908-09-17,17:18:00,"Fort Myer, Virginia",Military - U.S. Army,,Demonstration,Wright Flyer III,2.0,1.0,1.0,...,military,Wright Brothers,1.0,1.0,1908-01-01,September,Thursday,1900,0.0,1.0


In [6]:
df_mysql.to_csv(r'C:\Users\ingju\Desktop\proyect02\csv\air_accidents.csv', index=False, sep = ';')

In [11]:
# Importamos la librería necesaría
import mysql.connector

# Establecemos la conexión a MySQL
cnx = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Wip@8Wip@8'
)

# Instanciamos el cursor y creamos la base de datos 'air_flights' si no existe
cursor = cnx.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS air_flights")
cursor.execute("USE air_flights")

# Creamos la tabla 'air_accidents'
cursor.execute("""
    CREATE TABLE IF NOT EXISTS air_accidents (
        date DATE,
        time TIME,
        location VARCHAR(250),
        airline_operator VARCHAR(250),
        flight_no VARCHAR(250),
        route VARCHAR(250),
        aircraft_type VARCHAR(250),
        total_aboard INT,
        passengers_aboard INT,
        crew_aboard INT,
        total_fatalities INT,
        passengers_fatalities INT,
        crew_fatalities INT,
        summary VARCHAR(5000),
        country VARCHAR(150),
        surface VARCHAR(150),
        category VARCHAR(150),
        brand VARCHAR(150),
        surviving_crew INT,
        survivors INT,
        month VARCHAR(50),
        day VARCHAR(50),
        decade INT,
        mortality_rate FLOAT,
        survival_rate FLOAT
    )
""")

# Definimos los datos a utilizar a utilizar
data = df_mysql.to_dict('records')

# Escribimos la consulta SQL para insertar un solo registro
insert_query = """
    INSERT INTO air_accidents
    (date, time, location, airline_operator, flight_no, route, aircraft_type, total_aboard, passengers_aboard, crew_aboard, total_fatalities, passengers_fatalities, crew_fatalities, summary, country, surface, category, brand, surviving_crew ,survivors, month, day, decade, mortality_rate, survival_rate)
    VALUES
    (%(date)s, %(time)s, %(location)s, %(airline_operator)s, %(flight_no)s, %(route)s, %(aircraft_type)s, %(total_aboard)s, %(passengers_aboard)s, %(crew_aboard)s, %(total_fatalities)s, %(passengers_fatalities)s, %(crew_fatalities)s, %(summary)s, %(country)s, %(surface)s, %(category)s, %(brand)s, %(surviving_crew)s, %(survivors)s, %(month)s, %(day)s, %(decade)s, %(mortality_rate)s, %(survival_rate)s)
"""

# Ejecutamos la inserción de datos para cada registro
for row in data:
    # Reemplazar los valores NaN por None en el diccionario
    row = {key: None if pd.isna(value) else value for key, value in row.items()}
    cursor.execute(insert_query, row)

# Confirmamos los cambios en la base de datos
cnx.commit()

# Cerrar la conexión
cursor.close()
cnx.close()

In [12]:
# Cerrar la conexión nuevamente por si tenemos procesos activos
cursor.close()
cnx.close()